In [ ]:
!pip install plotnine
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from plotnine import *

pd.set_option('display.width', 120)

# Données sur les lycées

## Valeur ajoutée des lycées

https://data.education.gouv.fr/explore/dataset/fr-en-indicateurs-de-resultat-des-lycees-gt_v2/information/

Dans les données, il y a de nombreuses informations :
-   informations techniques sur les lycées (nom de l'établissement, département, région, secteur...)
-   effectifs des présents à l'examen du bac, selon les différentes filières
-   taux de réussite et de mention, selon les différentes filières
-   nombre de mentions détaillé, à l'échelle agrégée
-   indicateurs de valeur ajoutée pour le taux de réussite et pour le taux de mention

In [ ]:
va_data = pd.read_parquet("data/lycees_resultats.parquet")

va_data = va_data.assign(
    annee = va_data['annee'].astype(str).str[0:4].astype(int)
)

va_data = va_data.sort_values(["uai", "annee"]).reset_index()

va_data.columns

In [ ]:
va_data_0 = va_data[[
    'uai', 'annee', 'libelle_uai', 'code_region',
    'libelle_region', 'libelle_academie', 'code_departement',
    'libelle_departement', 'code_commune', 'libelle_commune', 'secteur'
    ]]

va_data_effectifs = va_data[[
    'uai', 'annee', 
    'eff_2nde', 'eff_1ere', 'eff_term', 
    'presents_total',
    'presents_l', 'presents_es', 'presents_s', 'presents_gnle', 'presents_sti2d', 'presents_std2a', 'presents_stmg', 'presents_stl', 'presents_st2s', 'presents_s2tmd', 'presents_sthr', 
    'nb_mentions_tb_avecf_g', 'nb_mentions_tb_sansf_g', 'nb_mentions_b_g', 'nb_mentions_ab_g', 
    'nb_mentions_tb_avecf_t', 'nb_mentions_tb_sansf_t', 'nb_mentions_b_t', 'nb_mentions_ab_t'
    ]]

va_data_acces = va_data[[
    'uai', 'annee', 
    'taux_acces_2nde', 'va_acces_2nde',
    'taux_acces_1ere', 'va_acces_1ere',
    'taux_acces_term', 'va_acces_term'
    ]]

va_data_reu = va_data[[
    'uai', 'annee', 
    'taux_reu_total', 'va_reu_total', 
    'taux_reu_l', 'taux_reu_es', 'taux_reu_s', 'taux_reu_gnle', 
    # 'taux_reu_sti2d', 'taux_reu_std2a', 'taux_reu_stmg', 'taux_reu_stl', 'taux_reu_st2s', 'taux_reu_s2tmd', 'taux_reu_sthr', 
    'va_reu_l', 'va_reu_es', 'va_reu_s', 'va_reu_gnle', 
    # 'va_reu_sti2d', 'va_reu_std2a', 'va_reu_stmg', 'va_reu_stl', 'va_reu_st2s', 'va_reu_s2tmd', 'va_reu_sthr'
    ]]

va_data_men = va_data[[
    'uai', 'annee', 
    'taux_men_l', 'taux_men_es', 'taux_men_s', 'taux_men_gnle', 'taux_men_sti2d', 'taux_men_std2a', 'taux_men_stmg', 'taux_men_stl', 'taux_men_st2s', 'taux_men_s2tmd', 'taux_men_sthr', 
    'va_men_l', 'va_men_es', 'va_men_s', 'va_men_gnle', 'va_men_sti2d', 'va_men_std2a', 'va_men_stmg', 'va_men_stl', 'va_men_st2s', 'va_men_s2tmd', 'va_men_sthr'
    ]]


In [ ]:
va_data_reu.head(10)

## Indices de position sociale

https://data.education.gouv.fr/explore/dataset/fr-en-ips_lycees/information/

L'indice de position sociale renseigne sur la composition sociale au sein des lycées.
En lien direct avec la théorie sociologie bourdieusienne, l'IPS propose donc de partir
de la PCS des parents d'élèves et d'en inférer une position sociale moyenne pour
l'ensemble de l'établissement. 

Plus l'IPS est grand, plus la position sociale des élèves de l'établissement est 
élévée, et plus les élèves sont supposés (sociologiquement) mieux réussir.

In [ ]:
ips_data = pd.read_parquet("data/lycees_ips.parquet")

ips_data = ips_data.assign(
    annee = ips_data['rentree_scolaire'].str[5:9].astype(int)
)

ips_data = ips_data.sort_values(["uai", "annee"]).reset_index()

ips_data.columns

In [ ]:
ips_data_0 = ips_data[[
    'uai', 'annee', 
    'nom_de_l_etablissment', 'secteur', 'type_de_lycee', 'academie', 
    'code_du_departement', 'departement', 
    'code_insee_de_la_commune', 'nom_de_la_commune'    
    ]]

ips_data_1 = ips_data[[
    'uai', 'annee', 
    'ips_voie_gt', 'ips_voie_pro', 'ips_ensemble_gt_pro', 
    'ecart_type_de_l_ips_voie_gt', 'ecart_type_de_l_ips_voie_pro'
    ]]

In [ ]:
ips_data_1.head(10)

## Jointure de deux sources

In [ ]:
# Jointure entre les deux
data = pd.merge(
    ips_data_1,
    va_data_reu,
    on = ['uai', 'annee'],
    how = 'inner'
)

In [ ]:
data.head()

In [ ]:
data.groupby(['annee']).count().iloc[:, 0]

In [ ]:
data.columns


In [ ]:
data['taux_reu_gnle'] = data['taux_reu_gnle'].fillna(0).astype(int)
data['taux_reu_gnle']

In [ ]:
(
    ggplot(
        data[data["taux_reu_gnle"].astype(int) > 0],
        aes(x = "taux_reu_gnle")
        ) +
    geom_density()
)

In [ ]:
data.dropna(subset=['taux_reu_gnle']).astype({'taux_reu_gnle': 'int'})

In [ ]:
(
    ggplot(
        data[data["taux_reu_gnle"].astype(int) > 10],
        aes(
            x = "ips_voie_gt",
            y = "taux_reu_gnle",
            color = "va_reu_total"
    )
  ) + 
  geom_point() + 
  geom_smooth(method = 'lm') +
  scale_color_gradient2(
    # palette = "RdYlGn",
    # breaks = [-5, -2, 2, 5],
    # limits = [-100, 100],
    # direction = 1
  ) +
    theme_bw()

)

# Données sur les bibliothèques

https://www.insee.fr/fr/statistiques/8217525?sommaire=8217537



In [ ]:
bpe_data = pd.read_parquet("data/bpe_equipements.parquet")

In [ ]:
bpe_data.head()

In [ ]:
biblio_data = bpe_data[bpe_data['TYPEQU'] == "F307"]
biblio_data = gpd.GeoDataFrame(
    biblio_data,
    geometry = gpd.points_from_xy(
        x = biblio_data['LAMBERT_X'], 
        y = biblio_data['LAMBERT_Y'], 
        crs = "EPSG:2154" 
    )
)


# Géolocalisation des lycées :

Pour fusionner les données biblio avec celles des lycées (calculer les distances entre lycées et biblios) , il fallait avoir des infos geo sur les localisations des lycées car le calcul de distances géodésiques nécessite des coordonnées précises. Or la base initiale des lycées ne contenait aucune information géographique (ni lat ni long )

Donc pour résoudre ce problème, on a identifié et importé une source complémentaire format geoJSON :

https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/export/?disjunctive.numero_uai&disjunctive.code_departement&disjunctive.code_region&disjunctive.code_academie&disjunctive.nature_uai&disjunctive.nature_uai_libe&disjunctive.code_commune&disjunctive.libelle_departement&disjunctive.libelle_region&disjunctive.libelle_academie&disjunctive.secteur_prive_code_type_contrat&disjunctive.secteur_prive_libelle_type_contrat&disjunctive.code_ministere&disjunctive.libelle_ministere

"Adresse et géolocalisation des établissements d'enseignement des premier et second degrés "

In [ ]:
geoloc_gdf = gpd.read_file("data/etablissements.geojson")
geoloc_gdf.head()


### Filtrer uniquement les lycées :

In [ ]:
geoloc_lycees = geoloc_gdf[
    geoloc_gdf["appellation_officielle"].str.contains("Lycée", case=False, na=False)
]

print("Nombre de lycées géolocalisés :", len(geoloc_lycees))



### Fusion entre les données lycées et géographiques :

In [ ]:
# Fusion sur le code uai
lycees_geo = data.merge(
    geoloc_lycees[["numero_uai", "geometry"]],
    left_on="uai",
    right_on="numero_uai",
    how="left"
)

# Transformation en GeoDataFrame
lycees_geo = gpd.GeoDataFrame(lycees_geo, geometry="geometry", crs="EPSG:4326")



In [ ]:
lycees_geo

In [ ]:
# Reprojection GeoDataFrames en Lambert-93 :

lycees_gdf = lycees_geo.to_crs("EPSG:2154")
biblios_gdf = biblio_data.to_crs("EPSG:2154")

### Merge spatial : associe chaque lycée à la bibliothèque la plus proche :

In [ ]:
lycees_proches = gpd.sjoin_nearest(
    lycees_gdf,
    biblio_data,
    how="left",
    distance_col="dist_proche_biblio_m"
)


In [ ]:
lycees_proches.describe()


### Nombre de bibliothèques situées dans différents rayons autour du lycée

In [ ]:
rayons = [50, 150, 300, 500 , 1000]

for r in rayons:
        
    # Créer un buffer autour de chaque lycée
    buffer_gdf = lycees_gdf[["uai", "geometry"]].copy()
    buffer_gdf["geometry"] = buffer_gdf.buffer(r)
    
    # Spatial join : bibliothèques dans le buffer
    join = gpd.sjoin(biblio_data, buffer_gdf, how="inner", predicate="within")
    
    # Compter le nombre de bibliothèques par lycée
    counts = join.groupby("uai").size().rename(f"nb_biblio_{r}")
    
    # Fusion avec le GeoDataFrame lycées
    lycees_proches = lycees_proches.merge(counts, left_on="uai", right_index=True, how="left")
    
    lycees_proches[f"nb_biblio_{r}"] = lycees_proches[f"nb_biblio_{r}"].fillna(0)


In [ ]:
lycees_proches.head()

In [ ]:
cols_utiles = [
    "uai", "annee",
    "ips_voie_gt", "ips_voie_pro", "ips_ensemble_gt_pro",
    "ecart_type_de_l_ips_voie_gt", "ecart_type_de_l_ips_voie_pro",
    "taux_reu_total", "va_reu_total", "taux_reu_l", "taux_reu_es",
    "taux_reu_s", "taux_reu_gnle",
    "LONGITUDE", "LATITUDE", "geometry",
    "dist_proche_biblio_m", 
    "nb_biblio_500", "nb_biblio_1000",
    "nb_biblio_2000", "nb_biblio_5000"
]

lycees_final = lycees_proches[cols_utiles].copy()


In [ ]:
lycees_final.describe()

In [ ]:
mq = lycees_final.isna().sum()
mq[mq > 0]